# Correction HLS Person Uri

HLS URLs has been put as being sameAs URI statements.

Right now it is:  Entity => same as URI (1943) => URI (967) => has value (1843) => Appellation

And it should be: Entity => same as external identifier (1842) => Identifier (41) => has value (1843) => Appellation

So we need to change the class of the inbetween entity, and the properties of the 2 statements.

Be ware, only for HLS URIs

In [1]:
# %load /home/gaetan/Desktop/geovpylib/templates/heading.py
%load_ext autoreload
%autoreload 2

# Common imports
import os
import pandas as pd, numpy as np
import datetime
#import time
#import json
#import requests
#import duckdb
#import plotly.express as px
# from multiprocessing import Pool

# Geovpylib library
import geovpylib.analysis as a
import geovpylib.database as db
import geovpylib.decorators as d
import geovpylib.importer as i
import geovpylib.magics
import geovpylib.pks as pks
import geovpylib.queries as q
import geovpylib.record_linkage as rl
import geovpylib.sparql as sparql
import geovpylib.utils as u
eta = u.Eta()

# Specific imports
# ...

# Global variables
# ...

# Connect to Geovistory database read mode
# db.connect_geovistory('prod')

# Connect to Geovistory database for insert
env = 'prod' # Database to query: "prod", "stag", "dev", "local"
pk_project = pks.projects.switzerland_and_beyond # The project to query/insert: integer
execute = True # Boolean to prevent to execute directly into databases
metadata_str = '' # kebab-lower-case or snake-lower-case. 
import_manner = 'one-shot' # 'one-shot' or 'batch'
db.connect_geovistory(env, pk_project, execute)
db.set_metadata({'import-id': datetime.datetime.today().strftime('%Y%m%d') + '-' + metadata_str})
db.set_insert_manner(import_manner)

# Connect to other database
# db_url_env_var_name = 'YELLOW_' # Name of an environment variable holding the Postgres database URL
# execute = False # Boolean to prevent to execute directly into databases
# db.connect_external(os.getenv(db_url_env_var_name), execute=False)

# Connect to a SPARQL endpoint
# sparql.connect_external('url')


[DB] Requests will not be executed
[DB] Connecting to PRODUCTION Database ... Connected!


# Fetch wrong datas

In [2]:
wrongs = db.query(f"""
    select
        s3.pk_entity as pk_stmt_{pks.properties.entity_sameAsURI_URI},
        r0.pk_entity as pk_uri,
        s1.pk_entity as pk_stmt_{pks.properties.appe_hasValue_string},
        a2.string as uri
    from information.resource r0
    inner join projects.info_proj_rel ipr0 on ipr0.fk_entity = r0.pk_entity and ipr0.fk_project = {pk_project} and ipr0.is_in_project = true
    inner join information.statement s1 on s1.fk_subject_info = r0.pk_entity and s1.fk_property = {pks.properties.appe_hasValue_string}
    inner join projects.info_proj_rel ipr1 on ipr1.fk_entity = s1.pk_entity and ipr1.fk_project = {pk_project} and ipr1.is_in_project = true
    inner join information.appellation a2 on a2.pk_entity = s1.fk_object_info
    inner join information.statement s3 on s3.fk_object_info = r0.pk_entity and s3.fk_property = {pks.properties.entity_sameAsURI_URI}
    inner join projects.info_proj_rel ipr3 on ipr3.fk_entity = s3.pk_entity and ipr3.fk_project = {pk_project} and ipr3.is_in_project = true
    where r0.fk_class = {pks.classes.uri}
""")
wrongs = wrongs[wrongs['uri'].str.contains('https://hls-dhs-dss.ch')]
wrongs.drop(columns=['uri'], inplace=True)

a.infos(wrongs)

Shape:  (23980, 3) - extract:


,pk_stmt_1943,pk_uri,pk_stmt_1843
11741,10704406,10656446,10728386
11744,10704476,10656516,10728456
11745,10704477,10656517,10728457
11746,10704478,10656518,10728458
11747,10704479,10656519,10728459


# Update wrong datas

In [4]:
stmts_1943 = u.get_sql_ready_str(wrongs['pk_stmt_1943'])
uris = u.get_sql_ready_str(wrongs['pk_uri'])
# stmts_1843 = u.get_sql_ready_str(wrongs['pk_stmt_1843'])

db.execute(f"""
    update information.statement set fk_property = 1842 where pk_entity in {stmts_1943};
    update information.resource set fk_class = 41 where pk_entity in {uris};
""")

[DB] SQL not executed
